In [0]:
import pandas as pd
import numpy as np
import nltk
import matplotlib.pyplot as plt
import re
from nltk.corpus import stopwords
import razdel
import maru
import zipfile
import warnings
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.svm import LinearSVC
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.svm import SVC
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from numpy import random

In [0]:
! pip install razdel
! pip install maru
! pip install xgboost

    100% |████████████████████████████████| 44.3MB 1.1MB/s 
    100% |████████████████████████████████| 13.9MB 3.6MB/s 
    100% |████████████████████████████████| 757kB 17.2MB/s 
    100% |████████████████████████████████| 51kB 14.7MB/s 
    100% |████████████████████████████████| 7.1MB 6.9MB/s 
    100% |████████████████████████████████| 256kB 26.4MB/s 
  Stored in directory: /root/.cache/pip/wheels/b7/ef/06/fbdd555907a7d438fb33e4c8675f771ff1cf41917284c51ebf
  Stored in directory: /root/.cache/pip/wheels/d4/88/d0/4e4abc83eb8f59a71e8dbd8ba99fd5615a3af1fac1ef7f8825
Successfully built lru-dict DAWG
torchvision 0.2.1 has requirement pillow>=4.1.1, but you'll have pillow 4.0.0 which is incompatible.
thinc 6.12.1 has requirement wrapt<1.11.0,>=1.10.0, but you'll have wrapt 1.11.1 which is incompatible.
pymc3 3.6 has requirement joblib<0.13.0, but you'll have joblib 0.13.2 which is incompatible.
featuretools 0.4.1 has requirement pandas>=0.23.0, but you'll have pandas 0.22.0 which is incomp

In [0]:
warnings.filterwarnings('ignore')

In [0]:
data, test = pd.read_csv("train.csv"), pd.read_csv('test.csv')

ТРАНСФОРМАЦИЯ

Дальше идут вспомогательные функции к главной - transform, которая преобразует исходный сырой текст в нужный формат (удаление лишних символов, нормализация и тд, там в функции все написано)

In [0]:
nltk.download('stopwords')
stopWords = set(stopwords.words('russian'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [0]:
def makeCorpus(vectors):
  corpus = []
  for vector in vectors:
    corpus.append(' '.join(vector))
  return corpus

In [0]:
def isShortnumber(number):
   return True if len(number)<=4 and number.isdigit() else False


In [0]:
def tokenize(text, stopWords):
    vectors = []
    a = re.compile(r'(?<=[А-Яа-я0-9])\.(?=[А-Яа-я])')
    b = re.compile(r'(?<=[А-Яа-я])\.(?=[А-Яа-я0-9])')
    c = re.compile(r'(?<=[А-Яа-я])\.(?=[A-Za-z])')
    d = re.compile(r'(?<=[A-Za-z])\.(?=[А-Яа-я])')
    for line in text:
        line = a.sub(' ', line)
        line = b.sub(' ', line)
        line = c.sub(' ', line)
        line = d.sub(' ', line)
        vectors.append([i.strip('. -') for i in line.split() if i.strip('. -') not in stopWords and not isShortnumber(i.strip('. -'))\
                      and len(i.strip('. -')) > 1])
    return vectors

In [0]:
def maru_lematize(text, analyzer):
  res_lem = []
  for line in text:
    res_lem.append([word.lemma for word in analyzer.analyze(line)])
  return res_lem

In [0]:
def transform(text):
  vectors = []
  
  #########lowercase#################
  text = text.apply(lambda x: x.lower())
  
  
  ###########delete \n################
  n = re.compile(r'\\n')
  text = text.apply(lambda x: n.sub(' ', x))
  
  
  ##############delete everything apart ё, -, \.##############
  pattern_mod = re.compile(r'[^a-zA-Z0-9а-яА-я ё.\-]')
  text = text.apply(lambda x: pattern_mod.sub(' ', x))
  
  
  ################carefully deleting \-#####################
  pattern_def = re.compile(r'(?<![A-Za-zА-Яа-я])\-(?![A-Za-zА-Яа-я])')
  text = text.apply(lambda x: pattern_def.sub(' ', x))
  
  
  #################delete short numbers#####################
  short_numbers = re.compile(r'(?<![0-9])(?<!\.)[0-9]{1,4}(?![0-9])(?!\.)')
  text = text.apply(lambda x: short_numbers.sub(r'', x))
  
  #################delete stopwords###################
  stopWords = set(stopwords.words('russian'))
  for line in text.index:
    text[line] = ' '.join([elem for elem in text[line].split() if elem not in stopWords])
    
    
  ###################tokenize######################
  text = tokenize(text, stopWords)
  
  
  ###################normalize######################
  analyzer = maru.get_analyzer(tagger='linear')
  text = maru_lematize(text, analyzer)
  
  
  ####################make corpus#####################
  corpus = makeCorpus(text)
  
  return corpus

После того, как преобразовали текст, векторизуем его

In [0]:
def make_tfidf_vectorizer(corpus):
  tfidf_vectorizer = TfidfVectorizer(tokenizer = lambda x: x.split())
  vectors = tfidf_vectorizer.fit_transform(corpus)
  return [vectors, tfidf_vectorizer]

ПОДГОТОВКА ДАННЫХ

In [0]:
X_train, X_test, y_train, y_test = train_test_split(data['text'], data['label'], test_size=0.3, random_state=42)

In [0]:
X_train, tfidfvectorizer = make_tfidf_vectorizer(transform(X_train))

In [0]:
X_test_corpus = transform(X_test)

In [0]:
X_test = tfidfvectorizer.transform(X_test_corpus)

МОДЕЛЬ

In [0]:
###########simple svc###################
model = LinearSVC(multi_class='ovr')

In [0]:
###############simple svc with balanced classes###############
model = LinearSVC(multi_class='ovr', class_weight='balanced')

In [0]:
###############Gradient Boosting with 1000 base algorithms##############
model = GradientBoostingClassifier(n_estimators = 1000)

In [0]:
#################Random forest with 500 trees################
model = RandomForestClassifier(n_estimators=500, random_state=42)

In [0]:
################bagging classifier svc balanced weights################
model = BaggingClassifier(base_estimator=LinearSVC(multi_class='ovr'), n_estimators=100)

In [0]:
##############bagging sgd classifier######################
model = BaggingClassifier(base_estimator=SGDClassifier(), n_estimators=10, max_samples=0.9, max_features=0.9)

In [0]:
############Boosting with SVC as initial#####################
class SVCmod:
    def fit(self, X, y, sample_weight=None):
        self.svc = LinearSVC().fit(X, y)
        return self.svc
    def predict(self, X):
        labels = self.svc.predict(X)
        one_hot_0 = np.array(labels == 0, dtype=np.float64)
        one_hot_1 = np.array(labels == 1, dtype=np.float64)
        one_hot_2 = np.array(labels == 2, dtype=np.float64)
        y = np.column_stack((one_hot_0, one_hot_1, one_hot_2))
        return y
one = SVCmod()
model = GradientBoostingClassifier(init=one)

In [0]:
################boosting with SVC initial #2 ######################
one = SVCmod()
model = GradientBoostingClassifier(init=one, n_estimators=10)

Это была попытка запустить ГБ из начального приближения с LinearSVC, который работает лучше всех. Я обернул класс LinearSVC, чтобы он возвращал предсказания в нужном формате, исходный класс не подходит, чтобы использовать его в бустинге на начальном приближении.  Не сработала идея

In [0]:
##############XGBoost###############
params = {'max_depth':[5, 8, 15], 'n_estimators':[50, 70, 150], 'booster':['gbtree', 'gblinear', 'dart']}
estimator = XGBClassifier()
model = GridSearchCV(estimator, params)

Дальше я пытаюсь найти шумовые объекты, для этого считаю отступы и удаляю те, на которых большой отрицательный отступ

In [0]:
########### find shum ###################

def sigmoid(a):
  return [1/(1 + np.exp(-elem)) for elem in a]

################ prepare all the data ##################
X, tfidf = make_tfidf_vectorizer(transform(data['text']))


############## fit linear estimator and make predictions #################
find_noise = LinearSVC()
find_noise.fit(X, data['label'])
labels_train_for_confidence = find_noise.predict(X)


################## extract confidence ############# это вероятности
probs_for_confidence = list(map(lambda x: sigmoid(x) , find_noise.decision_function(X)))


########### find margins ################## отступы нахожу, как вероятность умножить на 1, если класс правильный и на -1 в противном
otstups = []

for triple in zip(probs_for_confidence, labels_train_for_confidence, data['label']):
  otstups.append(max(triple[0]) if triple[1]==triple[2] else -max(triple[0]))
  
######### plot the distribution of margins ##########3

import matplotlib.pyplot as plt
%matplotlib inline

plt.hist(otstups, bins=10, edgecolor='b');

############### find noise indexes ################# Здесь еще устанавливаю порог, по которому считаю, что объект - шум

threshold = -0.7

indexes = []

for index in range(len(otstups)):
  if otstups[index]<threshold:
    indexes.append(index)
    
############ remove the noise ##############

X_train_no_noise = data.drop(indexes)

############### train the model on the data without noise ################

X_train_nn, tfidfvectorizer_nn = make_tfidf_vectorizer(transform(X_train_no_noise['text']))
X_test_nn = tfidfvectorizer_nn.transform(transform(test['text']))

######### fit the model ###################


model_no_noise = LinearSVC(class_weight='balanced')
model_no_noise.fit(X_train_nn, X_train_no_noise['label'])

########### make prediction ###############

predictions_no_noise = model_no_noise.predict(X_test_nn)


Поиск шума и попытка обучения как бы на данных без шумовых объектов не удалась, хотя шум действительно вроде получилось найти

Изначально получился очень большой словарь и число фичей почти сравнимо с числом объектов, поэтому бустинги могли плохо работать, поэтому я решил найти самые-самые фичи и использовать их для трэйна

In [0]:
############## feature importants ####################3

################## fit the gs tree ###################

tree = DecisionTreeClassifier()
params_first_it = {'max_depth':[i*2 for i in range(1, 15)]}
gs_first_it = GridSearchCV(tree, params_first_it)

gs_first_it.fit(X_train, y_train)


############# know the feature importance ###############
importance_first_it = gs_first_it.best_estimator_.feature_importances_

############### find influential ################## здесь извлекаются фичи, у которых скор важности не ноль

new_features_first_it = [elem[1] for elem in zip(importance_first_it, range(len(importance_first_it))) if elem[0]]

############# extract influential data ############# здесь данные преобразуются к набору данных на важных фичах
X_train_composed_1_it = X_train[:, new_features_first_it]
X_test_composed_1_it = X_test[:, new_features_first_it]

Несмотря на то, что у многих фичей был скор важности ноль, предсказательной способности в оставшихся фичах не достаточно, поэтому  я решил убрать из начального множества фичей самые важные, на оставшихся зафитить, и посчитать скор важности оставшихся, но в отсутсвие самых важных признаков

In [0]:
############### do one more iteration ######################

################ prepare data ########################
X_train_low = X_train[:, [i for i in range(X_train.shape[1]) if i not in new_features_first_it]]
X_test_low = X_test[:, [i for i in range(X_train.shape[1]) if i not in new_features_first_it]]


############## fit another gs tree ####################
tree = DecisionTreeClassifier()
params_sec_it = {'max_depth':[i*2 for i in range(1, 20)]}
gs_sec_it = GridSearchCV(tree, params_sec_it)

gs_sec_it.fit(X_train_low, y_train)

############## know the feature importance ###############
importance_sec_it = gs_sec_it.best_estimator_.feature_importances_

############### find influential ##################

new_features_sec_it = [elem[1] for elem in zip(importance_sec_it, range(len(importance_sec_it))) if elem[0]]

############# extract influential data #############
X_train_composed_2_it = X_train_low[:, new_features_sec_it]
X_test_composed_2_it = X_test_low[:, new_features_sec_it]

In [0]:
######### test after two iterations #################
from scipy.sparse import hstack
X_train_composed = hstack((X_train_composed_1_it, X_train_composed_2_it))
X_test_composed = hstack((X_test_composed_1_it, X_test_composed_2_it))
clf = LinearSVC()
clf.fit(X_train_composed, y_train)
res = clf.predict(X_test_composed)
print(classification_report(y_test, res))

              precision    recall  f1-score   support

           0       0.72      0.40      0.52      6499
           1       0.96      0.91      0.93     14882
           2       0.69      0.89      0.77     12360

   micro avg       0.80      0.80      0.80     33741
   macro avg       0.79      0.73      0.74     33741
weighted avg       0.81      0.80      0.80     33741



In [0]:
########### check another algorithm ####################

xgb = XGBClassifier(max_depth=28, n_estimators=500)
xgb.fit(X_train_composed, y_train)
res_ = xgb.predict(X_test_composed)
print(classification_report(y_test, res_))

              precision    recall  f1-score   support

           0       0.67      0.42      0.52      6499
           1       0.96      0.91      0.93     14882
           2       0.69      0.87      0.77     12360

   micro avg       0.80      0.80      0.80     33741
   macro avg       0.77      0.73      0.74     33741
weighted avg       0.81      0.80      0.79     33741



Это была попытка сократить размер данных при помощи поиска важных фичей, направление вроде правильное, то есть уже на 600 фичах вместо 60 тыс достигается 0.8, но это не совсем то, чего ожидалось

 ТЕСТИРОВАНИЕ МОДЕЛИ

In [0]:
model.fit(X_train, y_train)
result = clf.predict(X_test)

In [0]:
print('simple linear svc')
print(classification_report(y_test, result))

simple linear svc
              precision    recall  f1-score   support

           0       0.76      0.69      0.72      6499
           1       0.98      0.97      0.97     14882
           2       0.82      0.87      0.84     12360

   micro avg       0.88      0.88      0.88     33741
   macro avg       0.85      0.84      0.85     33741
weighted avg       0.88      0.88      0.88     33741



In [0]:
print('simple linear svc with balanced weights')
print(classification_report(y_test, result))

simple linear svc with balanced weights
              precision    recall  f1-score   support

           0       0.72      0.73      0.73      6499
           1       0.98      0.97      0.97     14882
           2       0.83      0.84      0.84     12360

   micro avg       0.87      0.87      0.87     33741
   macro avg       0.84      0.85      0.85     33741
weighted avg       0.88      0.87      0.88     33741



In [0]:
model.fit(X_train, y_train)
result = model.predict(X_test)
print('gradient boosting with 1000 basics')
print(classification_report(y_test, result))

gradient boosting with 1000 basics
              precision    recall  f1-score   support

           0       0.81      0.48      0.60      6499
           1       0.97      0.92      0.95     14882
           2       0.72      0.92      0.81     12360

   micro avg       0.84      0.84      0.84     33741
   macro avg       0.84      0.77      0.79     33741
weighted avg       0.85      0.84      0.83     33741



In [0]:
model.fit(X_train, y_train)
result = model.predict(X_test)
print('random forest with 500 trees')
print(classification_report(y_test, result))

random forest with 500 trees
              precision    recall  f1-score   support

           0       0.77      0.45      0.57      6499
           1       0.96      0.96      0.96     14882
           2       0.73      0.90      0.81     12360

   micro avg       0.84      0.84      0.84     33741
   macro avg       0.82      0.77      0.78     33741
weighted avg       0.84      0.84      0.83     33741



In [0]:
model.fit(X_train, y_train)
result = model.predict(X_test)
print('bagging on 100 svc')
print(classification_report(y_test, result))

bagging on 100 svc
              precision    recall  f1-score   support

           0       0.76      0.69      0.72      6499
           1       0.98      0.97      0.97     14882
           2       0.82      0.87      0.84     12360

   micro avg       0.88      0.88      0.88     33741
   macro avg       0.85      0.84      0.85     33741
weighted avg       0.88      0.88      0.88     33741



In [0]:
model.fit(X_train, y_train)
result = model.predict(X_test)
print('SGD classifier bagging 10')
print(classification_report(y_test, result))

SGD classifier bagging
              precision    recall  f1-score   support

           0       0.84      0.45      0.59      6499
           1       0.97      0.95      0.96     14882
           2       0.73      0.93      0.82     12360

   micro avg       0.85      0.85      0.85     33741
   macro avg       0.85      0.78      0.79     33741
weighted avg       0.86      0.85      0.84     33741



In [0]:
model.fit(X_train, y_train)
result = model.predict(X_test)
print('Boosting with initial SVC')
print(classification_report(y_test, result))

Boosting with initial SVC
              precision    recall  f1-score   support

           0       0.76      0.69      0.72      6499
           1       0.98      0.97      0.97     14882
           2       0.82      0.87      0.84     12360

   micro avg       0.88      0.88      0.88     33741
   macro avg       0.85      0.84      0.84     33741
weighted avg       0.88      0.88      0.88     33741



In [0]:
model.fit(X_train, y_train)
result = model.predict(X_test)
print('XGBoost')
print(classification_report(y_test, result))

XGBoost
              precision    recall  f1-score   support

           0       0.85      0.16      0.26      6499
           1       0.97      0.82      0.89     14882
           2       0.60      0.96      0.74     12360

   micro avg       0.75      0.75      0.75     33741
   macro avg       0.80      0.65      0.63     33741
weighted avg       0.81      0.75      0.71     33741



In [0]:
model.fit(X_train, y_train)
result = model.predict(X_test)
print(model.best_estimator_)
print('XGBoost with GS')
print(classification_report(y_test, result))

XGBClassifier(base_score=0.5, booster='gblinear', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=5, min_child_weight=1, missing=None, n_estimators=50,
       n_jobs=1, nthread=None, objective='multi:softprob', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)
XGBoost with GS
              precision    recall  f1-score   support

           0       0.79      0.65      0.71      6499
           1       0.98      0.96      0.97     14882
           2       0.81      0.89      0.85     12360

   micro avg       0.88      0.88      0.88     33741
   macro avg       0.86      0.84      0.84     33741
weighted avg       0.88      0.88      0.88     33741



ПОСЫЛКА

In [0]:
X_test_send_corpus = transform(test['text'])

In [0]:
X_test_send = tfidfvectorizer.transform(X_test_send_corpus)

In [0]:
result_send = model.predict(X_test_send)

In [0]:
with open('send.csv', 'w') as f:
    f.write('id,label\n')
    for i in range(len(result_send)):
        f.write(str(i) + ',' + str(result_send[i]) + '\n')

ОБУЧИТЬ СВМ НА ВСЕХ ДАННЫХ

In [0]:
X_train, tfidfvectorizer = make_tfidf_vectorizer(transform(data['text']))

In [0]:
#one = SVCmod()
#model = GradientBoostingClassifier(init=one, n_estimators=10)

In [0]:
model = LinearSVC()

In [0]:
model.fit(X_train, data['label'])

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)

In [0]:
X_test_send = tfidfvectorizer.transform(X_test_send_corpus)

In [0]:
result_send = model.predict(X_test_send)

In [0]:
with open('send.csv', 'w') as f:
    f.write('id,label\n')
    for i in range(len(result_send)):
        f.write(str(i) + ',' + str(result_send[i]) + '\n')

АУГМЕНТАЦИЯ ТЕКСТА

Попытка аугментировать текст. Было нескольно вариантов: симметричная перестановка слов, рандомное удаление, и замена рандомного слова на синоним. Последнее кажется наиболее адекватным, но вроде бы русского тезауруса нормального нет и откуда брать синонимы, я не знаю. Поэтому осталось два предыдущих варианта. Я генерировал для каждого предложения рандомный 0 или 1. 0 для удаления рандомного слова, 1 для симметричной перестановки. После если 0, рандомное слово удаляется, и предложение с той же меткой добавляется в данные, если 1, то я выбирал два слова и переставлял их местами, потом добавлял это предложение в данные. Ну, результата это не дало, но я сейчас понимаю, что нужно было аугментировать не все классы, а наверно только тот, на котором классификатор больше всего ошибался или самый маленький класс

In [0]:
################ augmentation function ########################

def augmentation(text):
  lists = []
  for line in text:
    lists.append(line.split())
  lists_copy = lists.copy()
  for sent in lists_copy:
    mode = random.randint(0, 2)
    length = len(sent)
    sent_copy = sent.copy()
    if not mode:
      if len(sent)<=1:
        lists.append(sent_copy)
      else:
        ind_1, ind_2 = random.choice(range(length), 2)
        from_ = sent[ind_1]
        sent_copy[ind_1] = sent_copy[ind_2]
        sent_copy[ind_2] = from_
        lists.append(sent_copy)
    if mode:
      if len(sent)<=1:
        lists.append(sent_copy)
      else:
        delete = random.randint(low=0, high = length)
        sent_copy.remove(sent_copy[delete])
        lists.append(sent_copy)
  texts = []
  for line in lists:
    texts.append(' '.join(line))
  return texts

In [0]:
############### augment the data ################

text_corpus = transform(data['text'])
text_aug = augmentation(text_corpus)
X_train_aug, tfidf_aug = make_tfidf_vectorizer(text_aug)

In [0]:
################ get appropriate target #################
target_aug = pd.concat((data['label'], data['label']))

In [0]:
################# fit the model ########################
model  = LinearSVC()
model.fit(X_train_aug, target_aug)

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)

In [0]:
############## prepare test data ##############
test_for_aug = tfidf_aug.transform(transform(test['text']))

In [0]:
##############make predictions ################
prediction_aug = model.predict(test_for_aug)

In [0]:
########## send solution #############
with open('send.csv', 'w') as f:
    f.write('id,label\n')
    for i in range(len(prediction_aug)):
        f.write(str(i) + ',' + str(prediction_aug[i]) + '\n')